# Práctica 2
## Parte 2. Recuperación de Información
<br>

__Alumnos:__
* __Frederick Ernesto Borges Noronha__
* __Victor Manuel Cavero Gracia__

Aquí vamos a utilizar el conjunto de datos de 20 Newsgroups que se encuentra disponible en Scikit-learn y que hemos usado en el notebook de ejemplo.

El conjunto consiste en textos de un foro sobre diferentes temas, desde hardware hasta religión. Algunos temas están muy relacionados, p.ej. "IBM PC hardware" y "Mac hardware", mientras que otros son más diversos, p.ej. "religion" o "hockey").

El objetivo de esta parte es poner en poner en práctica los conceptos de recuperación de información para realizar un buscador de mensajes en un foro.

### Apartado A

El conjunto está dividido de forma predeterminada en entrenamiento y prueba en porcentajes de 60 y 40, respectivamente (como se puede ver en el notebook de prueba). Usaremos únicamente la parte de entrenamiento como los mensajes a recuperar por nuestro buscador.

Vamos a utilizar una representación de la bolsa de palabras de **countVectorizer** con las siguientes opciones:
- La bolsa de palabras tendrá en cuenta la frecuencia de las palabras en cada mensaje (**binary=False**)
- Usa el diccionario que se encuentra en la siguiente URL y que ya usamos en el notebook de prueba. https://github.com/dwyl/english-words/blob/master/words.txt
- Usa la lista de palabras vacías (parámetro **stop_words**) que proporciona sklearn para el inglés
- Usando un rango de n-gramas de (1,1) (parámetro **ngram_range**).

Para calcular la similitud entre dos mensajes usaremos la similitud del coseno (**sklearn.metrics.pairwise.cosine_similarity**) que es capaz de medir la similitud entre los elementos (es decir, entre las filas) de dos matrices de vectores de términos pudiendo ser estas matrices densas o dispersas.

Toma 3 mensaje del conjunto de prueba para cada clase (es decir, para cada tema). Vas a usar cada uno de dichos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que más se parezcan a la consulta. Para ello sigue los siguientes pasos:

1. Usa la distancia del coseno entre el mensaje de consulta y los mensajes de entrenamiento.

2. Ordena los resultados de mayor a menor relevancia con la consulta.

3. Calcula la precisión de la lista de resultados con nivel de exhaustividad 3 y 10. 

    - La precisión a un nivel de exhaustividad X es el número de resultados que son relevantes (es decir, de la clase buscada) de entre los X primeros recuperados.

4. Calcula los valores de precisión media (para cada nivel de exhaustividad) para cada clase del conjunto de datos.

Se valorará el uso de funciones y la claridad del código, así como sus comentarios. Contesta a lo siguiente:

- ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?

- Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.

    - ¿Con qué clases se ha confundido más dicha consulta? 

    - ¿A qué crees que se deben los malos resultados?

Debes usar la parte de entrenamiento para construir la bolsa de palabras con frecuencia y bolsa de palabras con TF/IDF.

### Apartado B

Repite la secuencia de pasos descritos en el apartado a) pero ahora usa TF-IDF para ponderar el peso de los términos de la bolsa de palabras.  Para usar TF-IDF primero debes transformar los textos  usando  **countVectorizer** con **binary=False**  para  obtener  la  frecuencia  de  palabras  (exactamente igual que en el apartado anterior), y a continuación usar **TfidfTransformer** para modular dicha frecuencia según lo popular que sea cada término en el conjunto de mensajes de entrenamiento.

A continuación contesta a lo siguiente.

- ¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?

- Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.

### Imports:
A continuación se encuentran todos los imports de las librerias de las que haremos uso en esta práctica.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import operator

## Definición de Funciones

In [2]:
documentos_guardados = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
consultas_a_generar = fetch_20newsgroups(subset='test')

print("Saved documents:", len(documentos_guardados.data))
print("# of consults:", len(consultas_a_generar.data))

Saved documents: 11314
# of consults: 7532


In [3]:
# Crear el diccionario de palabras
with open('Datos/words.txt') as f:
    dictionary = f.read().splitlines()

print("Dictionary Length: ", len(dictionary), " words")

Dictionary Length:  466551  words


In [4]:
def obtenerNombresDeClase(target,consultas_a_generar):
    '''
    Obtiene una tupla con el ID de la clase y 
    el nombre de la clase a la que pertenece.
    '''
    return (target, consultas_a_generar.target_names[target])

In [5]:
def todosLosTemasTienenNElementos(temas, n):
    '''
    Comprueba que todos los temas tenga N cantidad de elementos.
    '''
    for key, value in temas.items():
        if (len(value) != n):
            return False
    return True

In [6]:
def buscarNMensajesPorTema(temas_consultas, mensajes_consultas, n):
    '''
    Genera un diccionario de la forma 
    {"Tema": ("ID_Mensaje", "Mensaje")}
    '''
    temas = {x: [] for x in consultas_a_generar.target_names}
    i = 0
    while (not todosLosTemasTienenNElementos(temas, n)):
        if (len(temas[temas_consultas[i]]) < n):
            value = (i, mensajes_consultas[i])
            temas[temas_consultas[i]].append(value)
        i += 1
    return temas

In [7]:
def recuperarNResultados(consulta,documentos, n):
    '''
    '''
    x = cosine_similarity(consulta,documentos)[0]
    y = [index for index in range(len(x))]
    values = list(zip(y,x))
    values.sort(reverse=True, key = operator.itemgetter(1))
    return values[0:n]

In [8]:
def calcularTargets(consulta, documentos, documentos_guardados, n):
    resultado = recuperarNResultados(consulta,documentos, n)
    ids, cosineSimilarity = zip(*resultado)
    targets = []
    for id in ids:
        targets.append(documentos_guardados.target[id])
    
    return targets, cosineSimilarity, ids

In [9]:
def precision(numRelevantDocs, numDocsObtained):
    return numRelevantDocs/numDocsObtained

In [10]:
def getNumRelevantDocs(claseBuscada, clasesObtenidas):
    count = 0
    for clase in clasesObtenidas:
        if (clase == claseBuscada):
            count += 1
    return count

In [11]:
def calcularPrecision(consulta, targetConsulta, documentos, documentos_guardados, n):
    targets, cosineSimilarity, ids = calcularTargets(consulta, documentos, documentos_guardados, n)
    ret = precision(getNumRelevantDocs(targetConsulta, targets),n)
    return ret, ids

In [12]:
def generarResultados(nMensajes, topics, documentos, documentos_guardados, ids, n):
    resultDict = dict()
    idsDict = dict()
    for key, values in nMensajes.items():
        resultDict[topics[values[0][0]]] = np.array([])
        idsDict[topics[values[0][0]]] = []
        for id, value in values:
            precision, ids_mensajes = calcularPrecision(value, ids[id], documentos, documentos_guardados, n)
            idsDict[topics[id]].append(ids_mensajes)
            resultDict[topics[id]] = np.append(resultDict[topics[id]], precision)
            
    return resultDict, idsDict

In [13]:
def imprimirResultado(resultados):
    print("Resultados Precisión\n")
    for key, value in resultados.items():
        print("{}: \n\t {} - Precisión = {}".format(key,value,value.mean()) )

In [14]:
def ejecutarBuscador(consultas_a_generar, consultas, documentos_guardados, documentos,n=3, numMensajes=3):
    id_and_topics = [obtenerNombresDeClase(x,consultas_a_generar) for x in consultas_a_generar.target]
    ids, topics = zip(*id_and_topics)
    nMensajes = buscarNMensajesPorTema(topics, consultas, n)
    resultados, idsDict = generarResultados(nMensajes, topics, documentos, documentos_guardados, ids, n)
    imprimirResultado(resultados)
    return resultados, idsDict

In [15]:
def comparar(resultado, resultadoTFIDF):
    columnas = ["Clase", "Mejora"]
    diccionario = {"Clase": [], "Mejora": []}
    for key, value in resultado.items():
        diccionario["Clase"].append(key)
        diccionario["Mejora"].append(resultadoTFIDF[key].mean() - value.mean())
    df = pd.DataFrame(data=diccionario)
    return df

## Apartado A

In [16]:
# Creamos el CountVectorizer

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english', ngram_range=(1,1), binary=False)
documentos=vectorizer.fit_transform(documentos_guardados.data)

In [17]:
# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
consultas=vectorizer.transform(consultas_a_generar.data)

In [18]:
resultadoEx3, idsEx3 = ejecutarBuscador(consultas_a_generar,consultas, documentos_guardados,documentos,3)

Resultados Precisión

alt.atheism: 
	 [1. 0. 0.] - Precisión = 0.3333333333333333
comp.graphics: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.os.ms-windows.misc: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.sys.ibm.pc.hardware: 
	 [1.         0.         0.66666667] - Precisión = 0.5555555555555555
comp.sys.mac.hardware: 
	 [0. 0. 0.] - Precisión = 0.0
comp.windows.x: 
	 [0.         0.33333333 0.66666667] - Precisión = 0.3333333333333333
misc.forsale: 
	 [0.66666667 0.66666667 1.        ] - Precisión = 0.7777777777777777
rec.autos: 
	 [0. 0. 1.] - Precisión = 0.3333333333333333
rec.motorcycles: 
	 [1.         1.         0.66666667] - Precisión = 0.8888888888888888
rec.sport.baseball: 
	 [1.         0.66666667 0.66666667] - Precisión = 0.7777777777777777
rec.sport.hockey: 
	 [0.33333333 0.66666667 1.        ] - Precisión = 0.6666666666666666
sci.crypt: 
	 [1.         0.33333333 0.33333333] - Precisión = 0.5555555555555555
sci.

In [20]:
resultadoEx10, idsEx10 = ejecutarBuscador(consultas_a_generar,consultas, documentos_guardados,documentos,10)

Resultados Precisión

alt.atheism: 
	 [0.7 0.  0.  0.  0.3 0.  0.2 0.2 0.1 0. ] - Precisión = 0.15
comp.graphics: 
	 [0.6 0.  0.  0.3 0.6 0.2 0.  0.  0.6 0.4] - Precisión = 0.26999999999999996
comp.os.ms-windows.misc: 
	 [0.8 0.1 0.  0.  0.1 1.  0.3 0.3 0.9 0.2] - Precisión = 0.37
comp.sys.ibm.pc.hardware: 
	 [1.  0.1 0.4 0.1 0.4 0.2 0.3 0.8 0.3 0.2] - Precisión = 0.38
comp.sys.mac.hardware: 
	 [0.1 0.1 0.  0.1 0.4 0.2 0.3 0.1 0.2 0. ] - Precisión = 0.15
comp.windows.x: 
	 [0.1 0.4 0.4 0.2 1.  0.5 0.5 0.2 0.1 0.1] - Precisión = 0.35000000000000003
misc.forsale: 
	 [0.5 0.6 0.6 0.8 0.9 0.7 0.2 0.5 0.  0.2] - Precisión = 0.5
rec.autos: 
	 [0.  0.  0.9 0.3 0.6 0.1 1.  0.9 0.2 0. ] - Precisión = 0.4
rec.motorcycles: 
	 [0.9 1.  0.3 1.  1.  0.7 1.  0.  1.  0.7] - Precisión = 0.76
rec.sport.baseball: 
	 [1.  0.8 0.5 0.8 0.  0.8 0.6 0.1 0.2 0.6] - Precisión = 0.5399999999999999
rec.sport.hockey: 
	 [0.2 0.6 0.9 1.  0.9 0.8 0.8 0.8 0.8 0.9] - Precisión = 0.77
sci.crypt: 
	 [0.8 0.2 0.1 0.6 0.3

**¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?**

Si exite una diferencia en los valores, esto puede deberse a que no hay suficientes datos comparables en el conjunto de entrenamiento para poder clasficar correctamente todas las clases presentes.

**Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.**

Para exhaustividad 3 las peores clases consultadas han sido: `comp.graphics`, `comp.os.ms-windows.misc` y `talk.politics.mideast` con una precisión media del 0.2222222222222222, pero con una exhaustividad de 10 se tiene como peor `talk.religion.misc` con una precision media de 0.16666666666666666.

A continuación mostramos los mensajes de cada clase en la que se ha fallado:

`INSERTE AQUI CODIGO DE IMPRIMIR MENSAJE`

**¿Con qué clases se ha confundido más dicha consulta?**

`RESPUESTA PENDIENTE`

**¿A qué crees que se deben los malos resultados?**

Esto sucede porque en el caso de exhaustividad 3 se comparar los 3 mensajes de cada clase con el top 3 mensajes obtenidos por el buscador, sin embargo con exhaustividad 10 se caparan los 3 mensajes con el top 10 dandonos así valores mas cercanos a la realidad de precisión media.

## Apartado B

In [21]:
# Creamos el CountVectorizer
tfidfer= TfidfTransformer()

# Calculamos el valor TF-IDF 
documentos_preprocessed=tfidfer.fit_transform(documentos)

In [22]:
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
consultas_preprocessed=tfidfer.transform(consultas)

In [23]:
resultadoEx3TFIDF, idsEx3TFIDF = ejecutarBuscador(consultas_a_generar,consultas_preprocessed, documentos_guardados,documentos_preprocessed,3)

Resultados Precisión

alt.atheism: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.graphics: 
	 [0.33333333 0.         0.        ] - Precisión = 0.1111111111111111
comp.os.ms-windows.misc: 
	 [0.33333333 0.         0.        ] - Precisión = 0.1111111111111111
comp.sys.ibm.pc.hardware: 
	 [1.         0.66666667 0.33333333] - Precisión = 0.6666666666666666
comp.sys.mac.hardware: 
	 [0.         0.33333333 0.66666667] - Precisión = 0.3333333333333333
comp.windows.x: 
	 [0.         0.66666667 0.33333333] - Precisión = 0.3333333333333333
misc.forsale: 
	 [0.33333333 0.66666667 0.33333333] - Precisión = 0.4444444444444444
rec.autos: 
	 [0.         0.33333333 1.        ] - Precisión = 0.4444444444444444
rec.motorcycles: 
	 [1.         1.         0.33333333] - Precisión = 0.7777777777777778
rec.sport.baseball: 
	 [1. 1. 1.] - Precisión = 1.0
rec.sport.hockey: 
	 [1. 1. 1.] - Precisión = 1.0
sci.crypt: 
	 [1.         0.33333333 0.33333333] - Precisión = 0.555555555555

In [24]:
resultadoEx10TFIDF, idsEx10TFIDF = ejecutarBuscador(consultas_a_generar,consultas_preprocessed, documentos_guardados,documentos_preprocessed,10)

Resultados Precisión

alt.atheism: 
	 [0.7 0.1 0.  0.  0.1 0.  0.4 0.1 0.  1. ] - Precisión = 0.24
comp.graphics: 
	 [0.6 0.3 0.  0.4 0.9 0.3 0.  0.  0.2 0.3] - Precisión = 0.3
comp.os.ms-windows.misc: 
	 [0.7 0.1 0.  0.  0.2 0.9 0.3 0.3 1.  0.5] - Precisión = 0.4
comp.sys.ibm.pc.hardware: 
	 [1.  0.6 0.1 0.5 0.6 0.3 0.3 0.8 0.6 0.2] - Precisión = 0.5
comp.sys.mac.hardware: 
	 [0.  0.2 0.4 0.  0.3 0.2 0.4 0.1 0.3 0. ] - Precisión = 0.19
comp.windows.x: 
	 [0.1 0.7 0.5 0.  1.  0.2 0.7 0.1 0.1 0.2] - Precisión = 0.36
misc.forsale: 
	 [0.5 0.5 0.2 0.8 0.8 0.7 0.1 0.1 0.1 0.2] - Precisión = 0.4
rec.autos: 
	 [0.2 0.4 0.9 0.5 0.9 0.  0.9 0.6 0.4 0.5] - Precisión = 0.53
rec.motorcycles: 
	 [1.  0.9 0.3 0.9 1.  0.9 1.  0.6 1.  0.6] - Precisión = 0.82
rec.sport.baseball: 
	 [1.  0.6 1.  0.9 0.1 0.7 0.8 0.1 1.  1. ] - Precisión = 0.72
rec.sport.hockey: 
	 [1.  1.  1.  0.9 1.  0.8 0.8 0.9 1.  1. ] - Precisión = 0.9400000000000001
sci.crypt: 
	 [0.9 0.1 0.1 1.  0.3 0.1 1.  1.  0.  1. ] - Precisió

**¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?**

Si han cambiado los valores, utilizando la función `comparar` podemos observar la diferencia entre los resultados de dos buscadores, en este caso observaremos la diferencia entre las precisiones medias donde si la columna mejora es positivo el resultado ha mejorado y viceversa.

### Comparación con Exhaustividad 3

In [25]:
comparar(resultadoEx3, resultadoEx3TFIDF)

,Clase,Mejora
0,alt.atheism,-0.111111
1,comp.graphics,-0.111111
2,comp.os.ms-windows.misc,-0.111111
3,comp.sys.ibm.pc.hardware,0.111111
4,comp.sys.mac.hardware,0.333333
5,comp.windows.x,0.000000
6,misc.forsale,-0.333333
7,rec.autos,0.111111
8,rec.motorcycles,-0.111111
9,rec.sport.baseball,0.222222


Como podemos observar en la tabla anterior, existen 5 clases que empeoran sus resultados, 6 clases que mantienen la misma precisión media y 9 clases que tienen una mejora en la precisión medio.

En general las busquedas con TF/IDF producen resultados positivos a pesar de que existen varias clases que se mantienen igual o incluso empeoran sus resultados.

In [26]:
comparar(resultadoEx10, resultadoEx10TFIDF)

,Clase,Mejora
0,alt.atheism,0.09
1,comp.graphics,0.03
2,comp.os.ms-windows.misc,0.03
3,comp.sys.ibm.pc.hardware,0.12
4,comp.sys.mac.hardware,0.04
5,comp.windows.x,0.01
6,misc.forsale,-0.10
7,rec.autos,0.13
8,rec.motorcycles,0.06
9,rec.sport.baseball,0.18


Como podemos observar en la tabla anterior, existen 2 clases que empeoran sus resultados, 2 clases que mantienen la misma precisión media y 16 clases que tienen una mejora en la precisión medio.

En general las busquedas con TF/IDF producen una mejora en los resultados obtenidos.

**Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.**

`RESPUESTA PENDIENTE`

In [ ]:
def imprimirConsulta(consultas_a_generar, id_consulta):
    print("Clase Consultada: {}\n\n".format(consultas_a_generar.target_names[id_consulta]))
    print("Mensaje Consultado: \n\n{}\n\n".format(consultas_a_generar.data[id_consulta])) 

In [ ]:
def imprimirResultados(documentos_guardados, id):
    print("Clase Obtenida: {}\n\n".format(consultas_a_generar.target_names[id]))
    print("Mensaje Obtenido: \n\n{}\n\n".format(consultas_a_generar.data[id])) 

In [ ]:
def imprimirMensajes(consultas_a_generar, id_consulta, documentos_guardados, id_resultados, comparador=min):
    imprimirConsultas(consultas_a_generar, id_consulta)
    for key, ids in id_resultados:
        imprimirResultados(documentos_guardados, comparador(id))